In [3]:
import fastai
from fastai.vision import *

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(42)

In [19]:
path = untar_data(URLs.MNIST)

### No Acc

In [28]:
data = ImageDataBunch.from_folder(path, train='training', valid='testing')
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy)
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,0.307696,0.191688,0.943000,00:13


In [30]:
data.batch_size

64

### Naive Acc 

In [31]:
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,3.926346,32.967186,0.148700,04:19


### Acc + BnFreeze

In [32]:
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32), BnFreeze])
learn.loss_func = CrossEntropyFlat(reduction='sum')
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,3.683803,100.270874,0.142800,04:16


### Increase Momentum 

In [33]:
def set_bn_mom(m:nn.Module, mom=0.9):
    "Set bn layers in eval mode for all recursive children of `m`."
    for l in m.children():
        if isinstance(l, bn_types):
            l.momentum = mom
        set_bn_mom(l, mom)

In [34]:
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')
set_bn_mom(learn.model, mom=0.9)
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,3.857730,41607.148438,0.067100,04:14


### Decrease Momentum

In [35]:
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')
set_bn_mom(learn.model, mom=0.01)
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,3.769473,55.299114,0.177100,04:13


### InstanceNorm

In [36]:
def bn2instance(bn):
    if isinstance(bn, nn.BatchNorm1d): inst = nn.InstanceNorm1d(bn.num_features, affine=True)
    elif isinstance(bn, nn.BatchNorm2d): inst = nn.InstanceNorm2d(bn.num_features, affine=True)
    elif isinstance(bn, nn.BatchNorm3d): inst = nn.InstanceNorm3d(bn.num_features, affine=True)
    
    inst.weight = bn.weight
    inst.bias = bn.bias
    inst.running_mean = nn.Parameter(bn.running_mean, requires_grad=False)
    inst.running_var = nn.Parameter(bn.running_var, requires_grad=False)
    inst.momentum = bn.momentum
    inst.eps = bn.eps
    inst.track_running_stats = bn.track_running_stats
    return (inst).to(bn.weight.device)

In [37]:
def convert_bn(list_mods, func=bn2instance):
    for i in range(len(list_mods)):
        if isinstance(list_mods[i], bn_types):
            list_mods[i] = func(list_mods[i])
        elif list_mods[i].__class__.__name__ in ("Sequential", "BasicBlock"):
            list_mods[i] = nn.Sequential(*convert_bn(list(list_mods[i].children()), func))
    return list_mods

In [38]:
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [39]:
learn.model = nn.Sequential(*convert_bn(list(learn.model.children()), bn2instance))

In [40]:
learn.freeze()

In [41]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,4.612559,nan,0.098000,06:41


### GroupNorm

In [42]:
def bn2group(bn):
    groupnorm = nn.GroupNorm(4, bn.num_features, affine=True)
    groupnorm.weight = bn.weight
    groupnorm.bias = bn.bias
    groupnorm.eps = bn.eps
    return (groupnorm).to(bn.weight.device)

In [43]:
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [44]:
learn.model = nn.Sequential(*convert_bn(list(learn.model.children()), bn2group))

In [45]:
gnorm = nn.GroupNorm(2,5)

In [46]:
learn.freeze()

In [47]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,0.252386,0.266318,0.958000,04:58


### Resnet + GroupNorm

In [48]:
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.resnet18, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [49]:
def change_all_BN(module):
    for i in range(5):
        atr = 'bn'+str(i)
        if hasattr(module, atr):
            setattr(module, atr, bn2group(getattr(module,atr)))

def wrap_BN(model):
    for i in range(len(model)):
        for j in range(len(model[i])):
            if isinstance(model[i][j], bn_types):
                model[i][j] = bn2group(model[i][j])
            elif model[i][j].__class__.__name__ == "Sequential":
                for k in range(len(model[i][j])):
                    if isinstance(model[i][j][k], bn_types):
                        model[i][j][k] = bn2group(model[i][j][k])
                    elif model[i][j][k].__class__.__name__ == "BasicBlock":
                        change_all_BN(model[i][j][k])
                        if hasattr(model[i][j][k],'downsample'):
                            if model[i][j][k].downsample is not None:
                                for l in range(len(model[i][j][k].downsample)):
                                     if isinstance(model[i][j][k].downsample[l], bn_types):
                                        model[i][j][k].downsample[l] = bn2group(model[i][j][k].downsample[l])
    

In [50]:
wrap_BN(learn.model)

In [51]:
learn.freeze()

In [ ]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time


### Resnet + GroupNorm (No Acc)

In [ ]:
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.resnet18, metrics=accuracy)

In [ ]:
wrap_BN(learn.model)
learn.freeze()
learn.fit(1)